In [1]:
import pandas as pd
import numpy as np

dataset_dir = '../../data/label.txt'
org = np.loadtxt(dataset_dir)
# org = pd.read_csv(dataset_dir, sep=' ',dtype={'balance':np.float64}, header=None)

In [5]:
import platform 
platform.architecture()  

('32bit', 'WindowsPE')

In [2]:
org.shape

(371751, 60)

In [2]:
test_set = org[-10:, :]

In [3]:
misscolidx = 6
test_set_pre = np.delete(test_set, misscolidx, axis=1)
# org_train = np.delete(org[:10000], misscolidx, axis=1)

In [4]:
import lightgbm as lgb
def LightGBM(X, y):
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
    params = {
        'boosting': 'gbdt',
        'objective': 'regression',
        'metric': 'rmse',
        'num_leaves': 31,
        'min_data_in_leaf': 20,
        'learning_rate': 0.015,
        'cat_smooth': 10,
        'feature_fraction': 0.8,
        'bagging_freq': 5,
        'verbosity': 0
    }
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=1000,
                    valid_sets=lgb_eval,
                    early_stopping_rounds=70)
    # 用得到的模型进行未知年龄结果预测
    predicted_feature = gbm.predict(test_X, num_iteration=gbm.best_iteration)
    print("---------best_iteration: ", gbm.best_iteration)
    return predicted_feature
LightGBM()

ImportError: No module named lightgbm

In [5]:
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
import random
def RandomForest(x,y,rf_iteration):
    rfr = RandomForestRegressor(random_state=0, n_estimators=rf_iteration,  n_jobs=-1)
    rfr.fit(x,y)
    del x,y
    return rfr

def LightGBM(X, y, gbm_iteration):
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
    params = {
        'boosting': 'gbdt',
        'objective': 'regression',
        'metric': 'rmse',
        'num_leaves': 31,
        'min_data_in_leaf': 20,
        'learning_rate': 0.015,
        'cat_smooth': 10,
        'feature_fraction': 0.8,
        'bagging_freq': 5,
        'verbosity': 0
    }
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=gbm_iteration,
                    valid_sets=lgb_eval,
                    early_stopping_rounds=70)
    return gbm

rand_num = 10000
rf_iteration = 2000
gbm_iteration = 1000
rand_t = random.sample(range(org.shape[0]-10), rand_num)
x = np.delete(org[rand_t],misscolidx,axis=1)
y = org[rand_t,misscolidx]

m1 = RandomForest(x,y,rf_iteration)
res1 = m1.predict(test_set)
print("****** RF result ******")
print("****** data_set={}, iterations={}".format(rand_num,rf_iteration))
print(res1)
print("****** L2 dist ******")
print(np.linalg.norm(res1 - org[-10:,misscolidx]))


m2 = LightGBM(x,y, gbm_iteration)
res2 = m2.predict(test_set)
print("****** LightGBM result ******")
print("****** data_set={}, iterations={}".format(rand_num,gbm_iteration))
print(res2)
print("****** L2 dist ******")
print(np.linalg.norm(res2 - org[-10:,misscolidx]))



MemoryError: 

In [6]:
print test_set_pre.shape
pre = m.predict(test_set_pre)
print pre
print org[-10:,misscolidx]

(10, 59)
[3016.07865625 2995.10821875 2981.98121875 2969.00540625 2960.70628125
 2949.569875   2945.0098125  2944.11915625 2947.92053125 4252.42178125]
[2983.75    2974.84375 2965.9375  2957.03125 2948.125   2939.21875
 2930.3125  2921.40625 2912.5     2903.59375]


In [35]:
import random
r = random.sample(range(100),5)

In [11]:
np.linalg.norm(res1 - org[-10:,misscolidx]) 

822.9341630404211

In [7]:
res2 = pre #5000 3000
np.linalg.norm(res2 - org[-10:,misscolidx]) 

205.69489300545845

In [7]:
res3 = pre
np.linalg.norm(res3 - org[-10:,misscolidx]) 

1350.350164027599